In [1]:
# Librerias

import os
import pandas as pd
import sys

# Añadir la raíz del proyecto al PYTHONPATH
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), "..")))


## Exploración de los dataset originales (enero a abril 2020)

In [ ]:
import os
import pandas as pd
from src.config import MESES_EVALUACION, TEST_RAW_PATH, TEST_PROCESSED_PATH
from src.data.dataset import load_dataset
from src.modeling.predict import load_model, evaluate_model

# Cargar modelo entrenado
model = load_model()

# Lista acumuladora
df_acum = []

for mes in MESES_EVALUACION:
    print(f"Procesando mes: {mes}")

    # Cargar datos crudos (100k)
    path_crudo = os.path.join(TEST_RAW_PATH, f"sample_raw_100k_{mes}.parquet")
    df_crudo = load_dataset(path_crudo)

    # Cargar datos procesados (100k)
    path_proc = os.path.join(TEST_PROCESSED_PATH, f"sample_processed_100k_{mes}.parquet")
    df_proc = load_dataset(path_proc)

    # Predecir
    _, _, df_pred = evaluate_model(df_proc, model)

    # Unir crudo + predicciones
    df_mes = pd.concat([
        df_crudo.reset_index(drop=True),
        df_pred[["true_label", "predicted_label", "probability"]].reset_index(drop=True)
    ], axis=1)

    df_mes["mes"] = mes
    df_acum.append(df_mes)

# Unir todos los meses
df_acum = pd.concat(df_acum, ignore_index=True)

# Mostrar las primeras filas
df_acum.head()
